## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-02-21-53-05 +0000,bbc,Google keeps browser but must share data with ...,https://www.bbc.com/news/articles/cp8zdrenm1zo...
1,2025-09-02-21-50-43 +0000,nyt,"Under New Law, Greece Can Imprison Rejected As...",https://www.nytimes.com/2025/09/02/world/europ...
2,2025-09-02-21-50-13 +0000,nypost,Google dodges forced selloff of Chrome browser...,https://nypost.com/2025/09/02/business/google-...
3,2025-09-02-21-47-48 +0000,bbc,Watch: Timelapse shows Northern Lights display...,https://www.bbc.com/news/videos/ckglgp9q2nno?a...
4,2025-09-02-21-43-20 +0000,nypost,Chilling photos show Bryan Kohberger’s bleak a...,https://nypost.com/2025/09/02/us-news/chilling...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2344/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
45,trump,47
47,china,15
8,new,15
108,he,12
305,military,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
75,2025-09-02-20-04-23 +0000,nypost,Would-be Trump assassin Ryan Routh will be on ...,https://nypost.com/2025/09/02/us-news/would-be...,99
256,2025-09-02-06-42-01 +0000,nypost,President Trump hints at ‘Biden Autopen’ portr...,https://nypost.com/2025/09/02/us-news/presiden...,96
296,2025-09-02-00-05-47 +0000,nypost,Bessent says Trump admin willing to hit Russia...,https://nypost.com/2025/09/01/world-news/besse...,95
171,2025-09-02-14-16-14 +0000,latimes,Trump deployment of military troops to Los Ang...,https://www.latimes.com/california/story/2025-...,94
11,2025-09-02-21-29-07 +0000,nypost,Would-be assassin Ryan Wesley Routh challenges...,https://nypost.com/2025/09/02/us-news/would-be...,94


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
75,99,2025-09-02-20-04-23 +0000,nypost,Would-be Trump assassin Ryan Routh will be on ...,https://nypost.com/2025/09/02/us-news/would-be...
256,49,2025-09-02-06-42-01 +0000,nypost,President Trump hints at ‘Biden Autopen’ portr...,https://nypost.com/2025/09/02/us-news/presiden...
289,44,2025-09-02-01-00-00 +0000,wsj,Disparate factions inside MAHA are sparring ov...,https://www.wsj.com/politics/policy/maha-kenne...
94,41,2025-09-02-19-04-50 +0000,nypost,‘Morning Joe’ host Joe Scarborough urges defia...,https://nypost.com/2025/09/02/us-news/joe-scar...
241,40,2025-09-02-08-51-33 +0000,bbc,Russia's Putin hails 'unprecedented' ties with...,https://www.bbc.com/news/articles/cr4e4ngvvnro...
166,37,2025-09-02-14-37-16 +0000,nypost,Venezuela President Nicolás Maduro claims US s...,https://nypost.com/2025/09/02/world-news/madur...
142,31,2025-09-02-16-46-17 +0000,bbc,UK borrowing costs hit 27-year high adding to ...,https://www.bbc.com/news/articles/cy989njnq2wo...
247,30,2025-09-02-08-23-36 +0000,nypost,"Sudan landslide kills over 1,000 people and wi...",https://nypost.com/2025/09/02/world-news/sudan...
307,30,2025-09-01-22-29-23 +0000,bbc,Watch: Buildings destroyed and rescue efforts ...,https://www.bbc.com/news/videos/cqxgx8d14wwo?a...
80,29,2025-09-02-19-58-20 +0000,nypost,EU antitrust regulators delay Google adtech fi...,https://nypost.com/2025/09/02/business/eu-anti...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
